<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Задача. Rope](#toc1_)    
    - [Формат входа.](#toc1_1_1_)    
    - [Формат выхода.](#toc1_1_2_)    
    - [Ограничения.](#toc1_1_3_)    
- [АВЛ-дерево (рекурсивное и в таком виде ограничено в производительности)](#toc2_)    
- [Сплей дерево](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Задача. Rope](#toc0_)

Ваша цель в данной задаче — реализовать структуру данных Rope. Данная структура данных хранит строку и позволяет эффективно вырезать кусок строки и переставить его в другое место.  
### <a id='toc1_1_1_'></a>[Формат входа.](#toc0_)

Первая строка содержит исходную строку S, вторая — число запросов q. Каждая из последующих q строк задаёт запрос тройкой чисел i, j, k и означает следующее: вырезать подстроку S[i..j] (где i и j индексируются с нуля) и вставить её после k-го символа оставшейся строки (где k индексируется с единицы), при этом если k = 0, то вставить вырезанный кусок надо в
начало.

### <a id='toc1_1_2_'></a>[Формат выхода.](#toc0_)

Выведите полученную (после всех q запросов) строку.

### <a id='toc1_1_3_'></a>[Ограничения.](#toc0_)

S содержит только буквы латинского алфавита. 1 ≤ |S| ≤ 300 000; 1 ≤ q ≤ 100 000; 0 ≤ i ≤ j ≤ n−1; 0 ≤ k ≤ n−(j−i+1).




# <a id='toc2_'></a>[АВЛ-дерево (рекурсивное и в таком виде ограничено в производительности)](#toc0_)
не прокатило...

In [11]:
class CharNode:
    __slots__ = ['char', 'size', 'left', 'right']
    def __init__(self, char):
        self.char = char
        self.size = 1
        self.left = None
        self.right = None

class RopeAVL():
    __slots__ = ['node', 'height', 'balance']
    def __init__(self):
        self.node = None
        self.height = -1
        self.balance = 0

    def __repr__(self):
        return repr(list(self.walk()))

    def add(self, char):                                                    # стартовую строку пихаем вправо
        if not self.node:
            self.node = CharNode(char) 
            self.node.left = RopeAVL() 
            self.node.right = RopeAVL() 
        else:
            self.node.right.add(char)
        self.rebalance()                                                    

    def pop(self):                                                          # извлечь правый узел для сплита на нем
        if self.node:
            if not self.node.right.node:                                    
                char = self.node.char
                self.node = self.node.left.node or None
                return char
            else:
                return self.node.right.pop()
            self.rebalance()                                                

    def rebalance(self):
        self.update(False)
        while self.balance < -1 or self.balance > 1: 
            if self.balance > 1:
                if self.node.left.balance < 0:  
                    self.node.left.lrotate() # we're in case II
                    self.update()
                self.rrotate()
                self.update()
                
            if self.balance < -1:
                if self.node.right.balance > 0:  
                    self.node.right.rrotate() # we're in case III
                    self.update()
                self.lrotate()
                self.update()

    def update(self, recurse=True):
        if self.node: 
            if recurse: 
                if self.node.left: 
                    self.node.left.update()
                if self.node.right:
                    self.node.right.update()
            self.height = max(self.node.left.height,
                              self.node.right.height) + 1 
            self.balance = self.node.left.height - self.node.right.height
            self.node.size = 1 + (self.node.left.node.size if self.node.left.node else 0) + \
                                 (self.node.right.node.size if self.node.right.node else 0)
        else: 
            self.height = -1 
            self.balance = 0

    def rrotate(self):                                                      # Rotate right pivoting on self 
        A = self.node 
        B = self.node.left.node 
        T = B.right.node 
        
        self.node = B 
        B.right.node = A 
        A.left.node = T 
    
    def lrotate(self):                                                      # Rotate left pivoting on self 
        A = self.node 
        B = self.node.right.node 
        T = B.left.node 
        
        self.node = B 
        B.left.node = A 
        A.right.node = T 

    def walk(self):
        node = self.node
        stack = list()
        while stack or node:
            if stack:
                node = stack.pop()
                yield node.char
                node = node.right.node
            while node:
                stack.append(node)
                node = node.left.node



def _merge_w_root(t1, t2, t):
    t.node.left = t1 if t1 else RopeAVL()
    t.node.right = t2 if t2 else RopeAVL()
    return t
    
def _avl_merge_w_root(t1, t2, t):
    t1h = t1.height
    t2h = t2.height
    if abs(t1h - t2h) <= 1:
        t = _merge_w_root(t1, t2, t)
        t.update()                                          # из-за этого полного апдейта все валится на глубине рекурсии
        return t
    elif t1h > t2h:
        Tx = _avl_merge_w_root(t1.node.right, t2, t)
        t1.node.right = Tx
        t1.rebalance()
        return t1
    else:
        Tx = _avl_merge_w_root(t1, t2.node.left, t)
        t2.node.left = Tx
        t2.rebalance()
        return t2

def split(t, k): # 1-based
    if not t.node:
        return RopeAVL(), RopeAVL()
    leftsize = t.node.left.node.size if t.node.left.node else 0
    if k < leftsize + 1:
        t1, t2 = split(t.node.left, k)
        t2x = _avl_merge_w_root(t2, t.node.right, t)    
        return t1, t2x
    else:
        t1, t2 = split(t.node.right, k - leftsize - 1)
        t1x = _avl_merge_w_root(t.node.left, t1, t)     
        return t1x, t2

def merge(t1, t2):
    if not t1.node:
        return t2
    elif not t2.node:
        return t1
    else:
        t = RopeAVL()
        t.node = CharNode(t1.pop())
        return _avl_merge_w_root(t1, t2, t)             


SAMPLES = "hlelowrold\n2\n1 1 2\n6 6 7", "abcdef\n2\n0 1 1\n4 5 0"
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)

rope = RopeAVL()

def splitter(i, j, k):
    # т.к. мы в теле функции не только читаем, но и присваиваем rope, то фактически мы создаем 
    # локальную переменную с таким именем, которая перекрывает глобальное имя, и получаем UnboundLocalError
    # чтобы этого избежать, объявляем ее как глобальную переменную. Если бы только читали, то можно было бы и не 
    # объявлять, ее бы и так нашли в глобальной области видимости
    global rope         

    l, r = split(rope, i)
    c, r = split(r, j - i + 1)
    print("".join(l.walk()), "".join(c.walk()), "".join(r.walk()))
    nl, nr = split(merge(l, r), k)
    print("".join(nl.walk()), "".join(c.walk()), "".join(nr.walk()))
    rope = merge(merge(nl, c), nr)
    print("".join(rope.walk()))
    print()

string = input()
n = int(input())

for ch in string:
    rope.add(ch)

for _ in range(n):
    splitter(*list(map(int, input().split())))

"".join(rope.walk())

h l elowrold
he l lowrold
hellowrold

hellow r old
hellowo r ld
helloworld



'helloworld'

# <a id='toc3_'></a>[Сплей дерево](#toc0_)

In [1]:
import sys

class CharNode:
    def  __init__(self, char):
        self.char = char
        self.size = 1
        self.parent = None
        self.left = None
        self.right = None

class RopeSplay:
    def __init__(self, root=None):
        if root:
            root.parent = None
        self.root = root
    
    def __repr(self):
        return " ".join(self.walk(func=lambda x: x.char))
    
    def __str__(self):
        return "".join(self.walk())

    # print the tree structure on the screen
    def pprint(self):
        if self.root:
            self.__print_helper(self.root, "", True)
        else:
            print("Empty rope")
    
    def __print_helper(self, currPtr, indent, last):
        if currPtr is not None:
            sys.stdout.write(indent)
            if last:
                sys.stdout.write("R---")
                indent += "     "
            else:
                sys.stdout.write("L---")
                indent += "|   "
            print(currPtr.char, currPtr.size)
            self.__print_helper(currPtr.left, indent, False)
            self.__print_helper(currPtr.right, indent, True)

    def add(self, char):
        node =  CharNode(char)
        y = None
        x = self.root
        while x:
            y = x
            x = x.right
        # y is parent of x
        node.parent = y
        if not y:
            self.root = node
        else:
            y.right = node
        # splay the node
        self.__splay(node)

    def update(self, x=None):
        x = x if x else self.root
        x.size = 1 + (x.right.size if x.right else 0) + \
                     (x.left.size if x.left else 0)
    
    # удалить самый правый, его значение вернуть
    def pop(self):
        x = self.root
        t = None 
        s = None
        if x:
            #find rightest
            x = self.right(x)
            # make top & split
            self.__splay(x)
            t = None
            s = x.left
            # del x
            char = x.char
            x = None
            # join
            if s:
                s.parent = None
            self.root = self.__join(s, t)
            return char

    # joins two trees s and t
    def __join(self, s, t):
        if s == None:
            return t
        if t == None:
            return s
        x = self.right(s)
        self.__splay(x)
        x.right = t
        t.parent = x
        self.update(x)
        return x

    # rotate left at node x
    def __left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != None:
            y.left.parent = x

        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y
        self.update(x)                      # х ниже поэтому первый
        self.update(y)                      
        

    # rotate right at node x
    def __right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right is not None:
            y.right.parent = x
        
        y.parent = x.parent;
        if x.parent is None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y 
        y.right = x
        x.parent = y
        self.update(x)                      # х ниже поэтому первый
        self.update(y) 
        

    # Splaying operation. It moves x to the root of the tree
    def __splay(self, x):
        while x.parent is not None:
            if x.parent.parent is None:
                if x == x.parent.left:
                    # zig rotation
                    self.__right_rotate(x.parent)
                else:
                    # zag rotation
                    self.__left_rotate(x.parent)
            elif x == x.parent.left and x.parent == x.parent.parent.left:
                # zig-zig rotation
                self.__right_rotate(x.parent.parent)
                self.__right_rotate(x.parent)
            elif x == x.parent.right and x.parent == x.parent.parent.right:
                # zag-zag rotation
                self.__left_rotate(x.parent.parent)
                self.__left_rotate(x.parent)
            elif x == x.parent.right and x.parent == x.parent.parent.left:
                # zig-zag rotation
                self.__left_rotate(x.parent)
                self.__right_rotate(x.parent)
            else:
                # zag-zig rotation
                self.__right_rotate(x.parent)
                self.__left_rotate(x.parent)

    # find the left node
    def left(self, x):
        while x.left:
            x = x.left
        return x

    # find the right node
    def right(self, x):
        while x.right:
            x = x.right
        return x

    def walk(self, func=lambda x: str(x.char)):
        x = self.root
        stack = list()
        while stack or x:
            if stack:
                x = stack.pop()
                yield func(x)
                x = x.right
            while x:
                stack.append(x)
                x = x.left
    
    def __merge_w_root(self, x, y, t):
        if x: x.parent = t
        if y: y.parent = t
        t.left = x
        t.right = y
        self.update(t)
        return t
    
    def __order_stat(self, k):  # 1-based
        x = self.root
        k_order = 1 + (x.left.size if x.left else 0)
        while x:
            if k < k_order:
                x = x.left
                k_order = k_order - 1 - (x.right.size if x.right else 0)
            elif k > k_order:
                x = x.right
                k_order = k_order + 1 + (x.left.size if x.left else 0)
            else:
                return x

    def split(self, k): # 1-based
        if k < 1:
            return RopeSplay(), RopeSplay(self.root)
        elif k > self.root.size - 1:
            return RopeSplay(self.root), RopeSplay()
        else:
            x = self.__order_stat(k)
            self.__splay(x)
            x, y = x, x.right 
            x.right = None
            self.update(x)
            if y: y.parent = None
            if y: self.update(y)
            return RopeSplay(x), RopeSplay(y)


class Splitter:
    def __init__(self):
        self.rope = RopeSplay()

    def add(self, char):
        self.rope.add(char)

    def __merge(self, t1: RopeSplay, t2: RopeSplay) -> RopeSplay:
        if t1.root is None:
            return t2
        elif t2.root is None:
            return t1
        else:
            t = RopeSplay()
            t.root = CharNode(t1.pop())
            if t1.root: t1.root.parent = t.root
            if t2.root: t2.root.parent = t.root
            t.root.left = t1.root
            t.root.right = t2.root
            t.update()
            return t

    def rearrange(self, i, j, k):
        l, r = self.rope.split(i)
        c, r = r.split(j - i + 1)
        nl, nr = self.__merge(l, r).split(k)
        self.rope = self.__merge(self.__merge(nl, c), nr)
        return self.rope

    
def main() -> RopeSplay:
    splitter = Splitter()   

    string = input()
    n = int(input())

    for ch in string:
        splitter.add(ch)

    for _ in range(n):
        [i, j, k] = list(map(int, input().split()))
        splitter.rearrange(i, j, k)
    
    return splitter.rope


SAMPLES = "hlelowrold\n2\n1 1 2\n6 6 7", "abcdef\n2\n0 1 1\n4 5 0", "abcdef\n3\n0 0 5\n4 4 5\n5 5 0"
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)

print(main())

helloworld
